# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.csv", sep=",", header=0)
publications


,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher
0,"Dennison, Jeffrey B; Sazhin, Daniel; Smith, Da...",Decision neuroscience and neuroeconomics: Rece...,Wiley Interdisciplinary Reviews: Cognitive Sci...,13.0,3.0,e1589,2022.0,Wiley Online Library
1,"Wyngaarden, James B; Johnston, Camille R; Sazh...",Corticostriatal responses to social reward are...,Social Cognitive and Affective Neuroscience,19.0,1.0,nsae033,2024.0,Oxford University Press UK
2,"Botvinik-Nezer, Rotem; Holzmeister, Felix; Cam...",Variability in the analysis of a single neuroi...,Nature,582.0,7810.0,84-88,2020.0,Nature Publishing Group UK London
3,"Dennison, Jeff B; Tepfer, Lindsey J; Smith, Da...",Tensorial independent component analysis revea...,Human Brain Mapping,44.0,7.0,2905-2920,2023.0,"John Wiley & Sons, Inc. Hoboken, USA"
4,"Sazhin, Daniel; Frazier, Angelique M; Haynes, ...",The role of social reward and corticostriatal ...,Journal of psychiatry and brain science,5.0,NaN,e200024,2020.0,NaN
5,"Chat, Iris Ka-Yi; Bart, Corinne; Dennison, Jef...",Inflammatory Signaling and Corticostriatal Fun...,Biological Psychiatry,89.0,9.0,S298-S299,2021.0,Elsevier
6,"Botvinik-Nezer, Rotem; Holzmeister, F; Camerer...",Variability in the analysis of a single neuroi...,NaN,NaN,NaN,NaN,2020.0,NaN
7,"Wyngaarden III, James B; Sharp, Cooper Joseph;...",Characterizing the Effects of Nyquist Ghosting...,NaN,NaN,NaN,NaN,2022.0,OSF
8,"Smith, David V; Ludwig, Rita M; Dennison, Jeff...",Social Reward Processing and Decision Making i...,NaN,NaN,NaN,NaN,2023.0,NaN
9,"Zaff, Ori; Wyngaarden, James B; Dennison, Jeff...",Social context and reward sensitivity enhance ...,bioRxiv,NaN,NaN,NaN,2023.0,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [18]:
publications

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher
0,"Dennison, Jeffrey B; Sazhin, Daniel; Smith, Da...",Decision neuroscience and neuroeconomics: Rece...,Wiley Interdisciplinary Reviews: Cognitive Sci...,13.0,3.0,e1589,2022.0,Wiley Online Library
1,"Wyngaarden, James B; Johnston, Camille R; Sazh...",Corticostriatal responses to social reward are...,Social Cognitive and Affective Neuroscience,19.0,1.0,nsae033,2024.0,Oxford University Press UK
2,"Botvinik-Nezer, Rotem; Holzmeister, Felix; Cam...",Variability in the analysis of a single neuroi...,Nature,582.0,7810.0,84-88,2020.0,Nature Publishing Group UK London
3,"Dennison, Jeff B; Tepfer, Lindsey J; Smith, Da...",Tensorial independent component analysis revea...,Human Brain Mapping,44.0,7.0,2905-2920,2023.0,"John Wiley & Sons, Inc. Hoboken, USA"
4,"Sazhin, Daniel; Frazier, Angelique M; Haynes, ...",The role of social reward and corticostriatal ...,Journal of psychiatry and brain science,5.0,NaN,e200024,2020.0,NaN
5,"Chat, Iris Ka-Yi; Bart, Corinne; Dennison, Jef...",Inflammatory Signaling and Corticostriatal Fun...,Biological Psychiatry,89.0,9.0,S298-S299,2021.0,Elsevier
6,"Botvinik-Nezer, Rotem; Holzmeister, F; Camerer...",Variability in the analysis of a single neuroi...,NaN,NaN,NaN,NaN,2020.0,NaN
7,"Wyngaarden III, James B; Sharp, Cooper Joseph;...",Characterizing the Effects of Nyquist Ghosting...,NaN,NaN,NaN,NaN,2022.0,OSF
8,"Smith, David V; Ludwig, Rita M; Dennison, Jeff...",Social Reward Processing and Decision Making i...,NaN,NaN,NaN,NaN,2023.0,NaN
9,"Zaff, Ori; Wyngaarden, James B; Dennison, Jeff...",Social context and reward sensitivity enhance ...,bioRxiv,NaN,NaN,NaN,2023.0,NaN


In [17]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    item.pub_date=item.year
    item.pub_date=item.Year
    year = item.pub_date
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

AttributeError: 'Series' object has no attribute 'pub_date'

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

 2009-10-01-paper-title-number-1.md
 2010-10-01-paper-title-number-2.md
 2015-10-01-paper-title-number-3.md
'2020-05-20-Variability in the analysis.md'
'2020-10-29-The Role of Social Reward ...in Substance Use.md'
'2021-05-01-Inflammatory Signaling.md'
'2022-02-08-Decision neuroscience and neuroeconomics:.md'
'2023-01-19-Substance use is related to differential activation and connectivity.md'
'2023-02-07-Tensorial Independent Component Analysis Reveals .md'


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Variability in the analysis of a single neuroimaging dataset by many teams"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2020-05-20
venue: 'Nature'
paperurl: 'http://jbdenniso.github.io/files/Nature-20_Variability.pdf'
citation: 'Botvinik-Nezer, R., Holzmeister, F., Camerer, C. F., Dreber, A., Huber, J., Johannesson, M., ... & Rieck, J. R. (2020). &quot;Variability in the analysis of a single neuroimaging dataset by many teams.&quot; <i>Nature </i>, 582(7810), 84-88.'
---
Data analysis workflows in many scientific domains have become increasingly complex and flexible. Here we assess the effect of this flexibility on the results of functional magnetic resonance imaging by asking 70 independent teams to analyse the same dataset, testing the same 9 ex-ante hypotheses1. The flexibility of analytical approaches is exemplified by the fact that no two teams 